1. Создаем таблицы по условию

create table public.customers(
	customer_ID integer primary key
	,first_name text
	,last_name text
	,gender text
	,DOB date 
	,job_title text
	,job_industry_category text
	,wealth_segment text not null
	,deceased_indicator text not null
	,owns_car text not null
	,address  varchar not null
	,postcode integer not null
	,state text not null
	,country text not null
	,property_valuation integer not null
);

create table public.products(
    id serial primary key
	,product_ID integer
	,brand text
	,product_line text
	,product_class text
	,product_size text
	,list_price varchar
	,standard_cost varchar
);

create table public.orders(
	order_ID integer primary key
	,customer_ID integer
	,order_date date
	,online_order varchar
	,order_status varchar
);

create table public.order_items(
	order_item_id integer primary key
	,order_ID integer
	,product_ID integer
	,quantity integer
	,item_list_price_at_sale integer
	,item_standard_cost_at_sale integer
);

2. Загружаем данные в таблицу customers

<img src="cust.png" width="600">

3. Загружаем данные в таблицу orders

<img src="order.png" width="600">

4. Загружаем данные в таблицу products

<img src="prod.png" width="600">

5. Загружаем данные в таблицу order_items

<img src="ord_it.png" width="600">

Далее выполняем запросы. 

-- 1 пункт
select p.job_industry_category, count(*) as customer_count
from public.customers p
group by p.job_industry_category 
order by customer_count desc;

<img src="dz3_1.png" width="600">

-- 2 пункт
select
	extract(year from o.order_date) as year,
	extract(month from o.order_date) as month,
	c.job_industry_category,
	sum(oi.quantity*oi.item_list_price_at_sale) as total_revenue
from public.orders o 
join public.order_items oi on o.order_ID = oi.order_ID
join public.customers c on o.customer_ID = c.customer_ID
where o.order_status = 'Approved'
group by year, month, c.job_industry_category
order by year, month, c.job_industry_category;

<img src="dz3_2.png" width="600">

-- 3 пункт
select
	p.brand,
	count(distinct case
		when c.job_industry_category = 'IT' and o.online_order = 'True' then o.order_id 
	end) as unique_online_orders
from public.products p
left join public.order_items oi on p.product_ID = oi.product_id 
left join public.orders o on oi.order_id = o.order_id 
left join public.customers c on o.customer_id = c.customer_id 
group by p.brand 
order by p.brand;

<img src="dz3_3.png" width="600">

-- 4 пункт
select
	c.customer_id,
	c.first_name,
	c.last_name,
	sum(oi.quantity * oi.item_list_price_at_sale) as total_revenue,
	max(oi.quantity * oi.item_list_price_at_sale) as max_order,
	min(oi.quantity * oi.item_list_price_at_sale) as min_order,
	count(*) as order_count,
	avg(oi.quantity * oi.item_list_price_at_sale) as avg_order
from public.customers c 
join public.orders o on c.customer_id = o.customer_id 
join public.order_items oi on o.order_id = oi.order_id
group by c.customer_id, c.first_name, c.last_name 
order by total_revenue desc, order_count desc

<img src="dz3_4.png" width="600">

-- 5 пункт
with customer_revenue as (
	select 
		c.customer_id,
		c.first_name,
		c.last_name,
		coalesce(sum(oi.quantity * oi.item_list_price_at_sale), 0) as total_revenue
	from public.customers c 
	left join public.orders o on c.customer_id = o.customer_id 
	left join public.order_items oi on o.order_id = oi.order_id 
	group by c.customer_id, c.first_name, c.last_name
),
top_max as (
	select * from customer_revenue
	order by total_revenue desc 
	limit 3
),
top_min as (
	select * from customer_revenue
	order by total_revenue asc
	limit 3
)
select * from top_max
union all 
select * from top_min 
order by total_revenue desc;

<img src="dz3_5.png" width="600">

-- 6 пункт
with ranked_orders as (
	select 
		c.customer_id,
		c.first_name,
		c.last_name,
		o.order_ID,
		o.order_date,
		row_number() over (partition by c.customer_id order by o.order_date) as rn
	from public.customers c 
	join public.orders o on c.customer_id = o.customer_id 
)
select
	customer_ID,
	first_name,
	last_name,
	order_ID,
	order_date
from ranked_orders
where rn = 2
order by order_date;


<img src="dz3_6.png" width="600">

-- 7 пункт
with order_dates as (
	select 
		c.customer_id,
		c.first_name,
		c.last_name,
		c.job_title,
		o.order_date,
		lag(o.order_date) over (partition by c.customer_id order by o.order_date) as prev_order_date
	from public.customers c 
	join public.orders o on c.customer_id = o.customer_id 
),
intervals as(
	select
		customer_ID,
		first_name,
		last_name,
		job_title,
		(order_date - prev_order_date) as gap_days
		from order_dates
		where prev_order_date is not null
)
select
	first_name,
	last_name,
	job_title,
	max(gap_days) as max_interval_days
from intervals
group by customer_id, first_name, last_name, job_title 
having count(*) >=1
order by max_interval_days desc;

<img src="dz3_7.png" width="600">

-- 8 пункт
with ranked_customers as(
	select 
		c.customer_id,
		c.first_name,
		c.last_name,
		c.wealth_segment,
		coalesce(SUM(oi.quantity * oi.item_list_price_at_sale), 0) as total_revenue,
		row_number() over (partition by c.wealth_segment order by sum(oi.quantity * oi.item_list_price_at_sale) desc nulls last) as rn
	from public.customers c
	left join public.orders o on c.customer_id = o.customer_id 
	left join public.order_items oi on o.order_id = oi.order_id 
	group by c.customer_id, c.first_name, c.last_name, c.wealth_segment 
)
select
	first_name,
	last_name,
	wealth_segment,
	total_revenue
from ranked_customers
where rn <= 5
order by wealth_segment, total_revenue desc;

<img src="dz3_8.png" width="600">